## Ml analysis

In [123]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import nibabel as nib
import os
import glob
from os.path import join as opj
from mpl_toolkits.axes_grid1 import ImageGrid
from unet3D import *
from torchsummary import summary
import monai
from monai.transforms import *
from mpl_toolkits.axes_grid1 import ImageGrid
import ssim
import tqdm

from sklearn.model_selection import train_test_split
import wandb
wandb.login()

from torchvision.utils import make_grid
import shutil
import pandas as pd

from nibabel.processing import smooth_image
import seaborn as sns

from scipy import stats
from nilearn.image import mean_img
from sklearn.metrics import r2_score
from scipy.stats import ttest_rel

In [14]:
dirs=["generated/original_final_mni","generated/original_smoothed_final_mni","generated/converted_final_mni"]

In [105]:
def compute_cnr(image, mask, box_shape=(3,3,4), N=1000):
    """
    Compute the CNR (contrast-to-noise ratio) metric of an image given a mask and box shape.

    Args:
    image: 3D numpy array, input image
    mask: 3D numpy array, mask with 1 for voxels inside the region of interest and 0 for voxels outside
    box_shape: tuple, shape of the box to sample (default is (3,3,4))
    N: int, number of times to sample boxes (default is 1000)

    Returns:
    cnr: float, computed CNR value
    """

    # initialize lists to store mean and std of boxes
    mean_inside = []
    mean_outside = []
    std_inside = []
    std_outside = []
    cnr_list=[]

    # iterate N times
    for i in tqdm.tqdm(range(N)):
        # sample two boxes inside the mask
        box1, box2 = None, None
        while box1 is None:
            # sample random indices
            x, y, z = np.random.randint(box_shape[0]//2, image.shape[0]-box_shape[0]//2, 1)[0], np.random.randint(box_shape[1]//2, image.shape[1]-box_shape[1]//2, 1)[0], np.random.randint(box_shape[2]//2, image.shape[2]-box_shape[2]//2, 1)[0]
            # check if both boxes are inside the mask
            if np.sum(mask[x-box_shape[0]//2:x+box_shape[0]//2+1, y-box_shape[1]//2:y+box_shape[1]//2+1, z-box_shape[2]//2:z+box_shape[2]//2+1]) == box_shape[0]*box_shape[1]*box_shape[2]:
                box1 = image[x-box_shape[0]//2:x+box_shape[0]//2+1, y-box_shape[1]//2:y+box_shape[1]//2+1, z-box_shape[2]//2:z+box_shape[2]//2+1]
                
        while box2 is None:
            x, y, z = np.random.randint(box_shape[0]//2, image.shape[0]-box_shape[0]//2, 1)[0], np.random.randint(box_shape[1]//2, image.shape[1]-box_shape[1]//2, 1)[0], np.random.randint(box_shape[2]//2, image.shape[2]-box_shape[2]//2, 1)[0]
            if np.sum(mask[x-box_shape[0]//2:x+box_shape[0]//2+1, y-box_shape[1]//2:y+box_shape[1]//2+1, z-box_shape[2]//2:z+box_shape[2]//2+1]) == box_shape[0]*box_shape[1]*box_shape[2]:
                box2 = image[x-box_shape[0]//2:x+box_shape[0]//2+1, y-box_shape[1]//2:y+box_shape[1]//2+1, z-box_shape[2]//2:z+box_shape[2]//2+1]

        # compute mean and std of boxes
        mean1, std1 = np.mean(box1, dtype=np.float64), np.std(box1, dtype=np.float64)
        mean2, std2 = np.mean(box2, dtype=np.float64), np.std(box2, dtype=np.float64)
        # store values in appropriate lists
        mean_inside.append(mean1)
        mean_outside.append(mean2)
        std_inside.append(std1)
        std_outside.append(std2)
        
        cnr= np.abs(mean1-mean2)/np.sqrt(std1**2+std2**2)
        cnr_list.append(cnr)
        
    return cnr_list

In [94]:
mask="/usr/local/fsl/data/standard/MNI152_T1_2mm_brain_mask.nii.gz"

mask=nib.load(mask)

In [95]:
compute_cnr(original_data[0],mask.get_fdata())

100%|███████████████████████████████████████| 1000/1000 [00:18<00:00, 53.30it/s]


-1.2258682840137798

In [15]:
original=[]
smoothed=[]
converted=[]
for subdir in os.listdir(dirs[0]):
    original+=glob.glob(os.path.join(dirs[0],subdir,"*.nii.gz"))
    smoothed+=glob.glob(os.path.join(dirs[1],subdir,"*.nii.gz"))
    converted+=glob.glob(os.path.join(dirs[2],subdir,"*.nii.gz"))
    

In [18]:
original=[nib.load(i) for i in original]
smoothed=[nib.load(i) for i in smoothed]
converted=[nib.load(i) for i in converted]


In [22]:
original_data=[]
smoothed_data=[]
converted_data=[]


for o,s,c in tqdm.tqdm(zip(original,smoothed,converted)):
    original_data.append(o.get_fdata())
    smoothed_data.append(s.get_fdata())
    converted_data.append(c.get_fdata())
    
original_data=np.array(original_data)
smoothed_data=np.array(smoothed_data)
converted_data=np.array(converted_data)


31it [00:00, 140262.59it/s]


In [23]:
outdir="generated/ml_metrics"
os.makedirs(outdir,exist_ok=True)

In [28]:
affine=original[0].affine

In [31]:
#mse

mse_original_converted=((original_data-converted_data)**2).mean(axis=0)
mse_smoothed_converted=((smoothed_data-converted_data)**2).mean(axis=0)

mse_original_converted_nii=nib.Nifti1Image(mse_original_converted,affine=affine)
nib.save(mse_original_converted_nii,os.path.join(outdir,"mse_original_converted.nii.gz"))

mse_smoothed_converted_nii=nib.Nifti1Image(mse_smoothed_converted,affine=affine)
nib.save(mse_smoothed_converted_nii,os.path.join(outdir,"mse_smoothed_converted.nii.gz"))


In [32]:
#normdiff

normdiff_original_converted= ((original_data-converted_data)/(original_data+converted_data)).mean(axis=0)
normdiff_smoothed_converted= ((smoothed_data-converted_data)/(smoothed_data+converted_data)).mean(axis=0)

normdiff_original_converted_nii=nib.Nifti1Image(normdiff_original_converted,affine=affine)
nib.save(normdiff_original_converted_nii,os.path.join(outdir,"normdiff_original_converted.nii.gz"))

normdiff_smoothed_converted_nii=nib.Nifti1Image(normdiff_smoothed_converted,affine=affine)
nib.save(normdiff_smoothed_converted_nii,os.path.join(outdir,"normdiff_smoothed_converted.nii.gz"))



/tmp/ipykernel_2674726/3732568015.py:3: RuntimeWarning: invalid value encountered in true_divide
  normdiff_original_converted= ((original_data-converted_data)/(original_data+converted_data)).mean(axis=0)
/tmp/ipykernel_2674726/3732568015.py:4: RuntimeWarning: invalid value encountered in true_divide
  normdiff_smoothed_converted= ((smoothed_data-converted_data)/(smoothed_data+converted_data)).mean(axis=0)


## CNR metric

In [106]:
cnr_original=[compute_cnr(i,mask.get_fdata()) for i in original_data]
cnr_smoothed=[compute_cnr(i,mask.get_fdata()) for i in smoothed_data]
cnr_converted=[compute_cnr(i,mask.get_fdata()) for i in converted_data]

 28%|███████████▎                            | 283/1000 [00:05<00:12, 56.95it/s]/tmp/ipykernel_2674726/3731848108.py:47: RuntimeWarning: invalid value encountered in double_scalars
  cnr= np.abs(mean1-mean2)/np.sqrt(std1**2+std2**2)
100%|███████████████████████████████████████| 1000/1000 [00:18<00:00, 54.29it/s]


In [107]:
np.savetxt("cnr_original_dist.txt",np.array(cnr_original))
np.savetxt("cnr_smoothed_dist.txt",np.array(cnr_smoothed))
np.savetxt("cnr_converted_dist.txt",np.array(cnr_converted))


1000

In [143]:
pval_original_converted=[ttest_rel(cnr_original[i],cnr_converted[i])[-1] for i in range(len(cnr_original))]
pval_smoothed_converted=[ttest_rel(cnr_smoothed[i],cnr_converted[i])[-1] for i in range(len(cnr_original))]
pval_original_smoothed=[ttest_rel(cnr_original[i],cnr_smoothed[i])[-1] for i in range(len(cnr_original))]

In [149]:
df_pvalues=pd.DataFrame(np.array([pval_original_converted,pval_smoothed_converted,pval_original_smoothed]).T, columns=["original_converted_p","smoothed_converted_p", "original_smoothed_p"])

In [150]:
df_pvalues

,original_converted_p,smoothed_converted_p,original_smoothed_p
0,8.784801e-26,3.781214e-11,2.462387e-08
1,1.495617e-10,3.521443e-07,2.329752e-02
2,1.843970e-06,3.558294e-01,1.583964e-06
3,8.826152e-11,8.882254e-05,3.296660e-04
4,2.980862e-18,2.283485e-17,5.275266e-01
5,2.355274e-10,1.695274e-05,8.244829e-03
6,1.443307e-06,6.511213e-03,1.453418e-02
7,4.061235e-15,7.206151e-10,2.048052e-02
8,NaN,NaN,NaN
9,NaN,NaN,3.252454e-02


In [ ]:
## lavorare qui, fare contronto tra mediane!

In [126]:
cnr_original_medians=np.percentile(np.array(cnr_original),50,axis=1)
cnr_smoothed_medians=np.percentile(np.array(cnr_smoothed),50,axis=1)
cnr_converted_medians=np.percentile(np.array(cnr_converted),50,axis=1)


cnr_original_medians = cnr_original_medians[~np.isnan(cnr_original_medians)]
cnr_smoothed_medians = cnr_smoothed_medians[~np.isnan(cnr_smoothed_medians)]
cnr_converted_medians = cnr_converted_medians[~np.isnan(cnr_converted_medians)]

In [135]:
cnr_smoothed_medians=cnr_smoothed_medians[:29]

In [136]:
_,pval_original_converted=ttest_rel(cnr_original_medians,cnr_converted_medians)
_,pval_smoothed_converted=ttest_rel(cnr_smoothed_medians,cnr_converted_medians)
_,pval_original_smoothed=ttest_rel(cnr_original_medians,cnr_smoothed_medians)

print(f"p value between original and converted: {pval_original_converted}")
print(f"p value between smoothed and converted: {pval_smoothed_converted}")
print(f"p value between original and smoothed: {pval_original_smoothed}")

p value between original and converted: 6.630426977695666e-08
p value between smoothed and converted: 0.0001325920802863473
p value between original and smoothed: 0.011215717733506474


In [34]:
atlas_path="HarvardOxford/HarvardOxford-sub-maxprob-thr25-2mm.nii.gz"
atlas_indices_path="HarvardOxford-Subcortical.csv"

In [35]:
df=pd.read_csv(atlas_indices_path)
df["label"]=df.index+1
df

left_thal_label=4
rigth_thal_label=15
atlas=nib.load(atlas_path).get_fdata()

In [61]:
mse=[]
mse_smoothed=[]
normdiff=[]
normdiff_smoothed=[]
cnr=[]
cnr_smoothed=[]
perc=[]
perc_smoothed=[]

x_mse=((original_data-converted_data)**2)
x_mse_smoothed=((smoothed_data-converted_data)**2)

x_normdiff=((original_data-converted_data)/(original_data+converted_data))
x_normdiff_smoothed=((smoothed_data-converted_data)/(smoothed_data+converted_data))

for label in tqdm.tqdm(df.label):
    mse.append(np.median(x_mse[:,atlas==label],axis=-1).mean())
    mse_smoothed.append(np.median(x_mse_smoothed[:,atlas==label],axis=-1).mean())
    
    normdiff.append(np.median(x_normdiff[:,atlas==label],axis=-1).mean())
    normdiff_smoothed.append(np.median(x_normdiff_smoothed[:,atlas==label],axis=-1).mean())
    
    #cnr
    roi_orig=original_data[:,atlas==label]
    roi_smooth=smoothed_data[:,atlas==label]
    roi_converted=converted_data[:,atlas==label]
    
    perc_values=(roi_orig.mean(axis=-1)-roi_converted.mean(axis=-1))/roi_orig.mean(axis=-1)
    perc_smoothed_values=(roi_smooth.mean(axis=-1)-roi_converted.mean(axis=-1))/roi_smooth.mean(axis=-1)
    
    cnr_values=(roi_orig.mean(axis=-1)-roi_converted.mean(axis=-1))/(np.concatenate((roi_orig,roi_converted),axis=-1).std(axis=-1))
    cnr_smoothed_values=(roi_smooth.mean(axis=-1)-roi_converted.mean(axis=-1))/(np.concatenate((roi_smooth,roi_converted),axis=-1).std(axis=-1))
    
    cnr.append(cnr_values.mean())
    cnr_smoothed.append(cnr_smoothed_values.mean())
                                                                                 
    perc.append(perc_values.mean())
    perc_smoothed.append(perc_smoothed_values.mean())
                                                                            

/tmp/ipykernel_2674726/959636557.py:13: RuntimeWarning: invalid value encountered in true_divide
  x_normdiff=((original_data-converted_data)/(original_data+converted_data))
/tmp/ipykernel_2674726/959636557.py:14: RuntimeWarning: invalid value encountered in true_divide
  x_normdiff_smoothed=((smoothed_data-converted_data)/(smoothed_data+converted_data))
100%|███████████████████████████████████████████| 21/21 [00:01<00:00, 13.26it/s]


In [63]:
df["mse"]=mse
df["mse_smoothed"]=mse_smoothed

df["normdiff"]=normdiff
df["normdiff_smoothed"]=normdiff_smoothed

df["cnr"]=cnr
df["cnr_smoothed"]=cnr

df["perc"]=perc
df["perc_smoothed"]=perc_smoothed

In [137]:
df

,index,region,label,mse,mse_smoothed,normdiff,normdiff_smoothed,cnr,cnr_smoothed,perc,perc_smoothed
0,0,Left Cerebral White Matter,1,0.002043,0.001864,NaN,NaN,-0.247115,-0.247115,-0.138468,-0.137682
1,1,Left Cerebral Cortex,2,0.002175,0.002010,NaN,NaN,-0.020036,-0.020036,-0.060398,-0.061981
2,2,Left Lateral Ventricle,3,0.001080,0.000957,-0.048741,-0.039625,-0.145409,-0.145409,-0.090668,-0.083651
3,3,Left Thalamus,4,0.003785,0.003605,-0.014023,-0.011635,0.052091,0.052091,-0.053337,-0.055479
4,4,Left Caudate,5,0.001715,0.001565,-0.033413,-0.030492,-0.080926,-0.080926,-0.076930,-0.079563
5,5,Left Putamen,6,0.002840,0.002670,-0.049212,-0.045996,-0.185363,-0.185363,-0.131171,-0.131096
6,6,Left Pallidum,7,0.002929,0.002752,-0.027579,-0.025284,-0.011623,-0.011623,-0.086704,-0.087702
7,7,Brain-Stem,8,0.002770,0.002544,NaN,-0.036729,-0.085208,-0.085208,-0.078196,-0.079295
8,8,Left Hippocampus,9,0.002698,0.002507,-0.047704,-0.044689,-0.186292,-0.186292,-0.113026,-0.113367
9,9,Left Amygdala,10,0.003107,0.002899,-0.051349,-0.049035,-0.214438,-0.214438,-0.128823,-0.130244


In [69]:
df.to_csv("ml_metrics.csv")

In [68]:
df.median() 

/tmp/ipykernel_2674726/530051474.py:1: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.median()


index                10.000000
label                11.000000
mse                   0.002770
mse_smoothed          0.002544
normdiff             -0.036102
normdiff_smoothed    -0.032365
cnr                  -0.083008
cnr_smoothed         -0.083008
perc                 -0.086704
perc_smoothed        -0.086454
dtype: float64